# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:

# plotting
import matplotlib.pyplot as plt

# data cleanup and math
import pandas as pd
import numpy as np
from scipy.stats import linregress

# API requests
import time
import json
import requests

#API key
from api_keys import weather_api_key

# Determine city based on latitude and longitude
from citipy import citipy

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


622

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [3]:
# reinitializes dataframe
worldWeather_df = pd.DataFrame()


In [4]:
print("Beginning Weather Data Extraction:")
for i, city in enumerate(cities):
    try:
        apiCall = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}'
        apiResponse =  requests.get(apiCall)
        jsonWeather = apiResponse.json()
        cityWeather_df = pd.DataFrame(
            {
                "City": city,
                'Longitude': jsonWeather['coord']['lon'],
                'Latitude': jsonWeather['coord']['lat'],
                'Temperature (°c)': jsonWeather['main']['temp']-273.15,
                'Max_Temperature (°c)': jsonWeather['main']['temp_max']-273.15,
                'Humidity': jsonWeather['main']['humidity'],
                'Clouds': jsonWeather['clouds']['all'],
                'Wind Speed': jsonWeather['wind']['speed'],
                'Country': jsonWeather['sys']['country'],
                'Datetime': jsonWeather['dt']
            }, index=[i]
        )

        worldWeather_df = worldWeather_df.append(cityWeather_df)
        print(f'{i} {city} successfully loaded.')

    except:
         print(f'Unable to load {city} data.')
        
print('Extraction Complete.')
    

0 pimentel successfully loaded.
1 hermanus successfully loaded.
Unable to load sentyabrskiy data.
3 lebu successfully loaded.
4 busselton successfully loaded.
5 hobart successfully loaded.
6 rikitea successfully loaded.
7 saint-philippe successfully loaded.
8 walvis bay successfully loaded.
9 mataura successfully loaded.
10 talnakh successfully loaded.
11 bluff successfully loaded.
12 souillac successfully loaded.
13 aklavik successfully loaded.
14 avarua successfully loaded.
15 enterprise successfully loaded.
16 esperance successfully loaded.
Unable to load bargal data.
18 hammerfest successfully loaded.
19 yellowknife successfully loaded.
20 los llanos de aridane successfully loaded.
Unable to load qurbah data.
22 sur successfully loaded.
23 alice springs successfully loaded.
24 cayenne successfully loaded.
25 sembe successfully loaded.
Unable to load marcona data.
27 ranong successfully loaded.
28 solnechnyy successfully loaded.
29 albany successfully loaded.
Unable to load tuggurt 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [6]:
worldWeather_df.to_csv('./output_data/worldWeather.csv')

In [8]:
display(worldWeather_df)

,City,Longitude,Latitude,Temperature (°c),Max_Temperature (°c),Humidity,Clouds,Wind Speed,Country,Datetime
0,pimentel,-79.9342,-6.8367,17.78,17.78,85,0,1.34,PE,1619747019
1,hermanus,19.2345,-34.4187,10.00,10.00,92,58,1.54,ZA,1619746906
3,lebu,-73.6500,-37.6167,12.81,12.81,95,100,3.57,CL,1619746911
4,busselton,115.3333,-33.6500,12.92,13.33,92,9,0.45,AU,1619747021
5,hobart,147.3294,-42.8794,17.59,18.89,72,20,4.12,AU,1619746821
...,...,...,...,...,...,...,...,...,...,...
617,sri aman,111.4621,1.2376,24.00,24.00,100,40,0.61,MY,1619747391
618,gwanda,29.0000,-20.9333,16.46,16.46,65,44,2.18,ZW,1619747391
619,bathsheba,-59.5167,13.2167,25.00,25.00,61,20,4.63,BB,1619747392
620,manitouwadge,-85.8403,49.1215,5.00,5.00,70,51,1.54,CA,1619747393


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression